# 🚀 Referencias Circulares y anidadas en Pydantic
Referencias Circulares en Pydantic v2
Las referencias circulares son un patrón común en muchas estructuras de datos, donde un modelo hace referencia a otro que, directa o indirectamente, vuelve a hacer referencia al primero. Pydantic v2 permite manejar estas estructuras mediante el uso de nombres de tipo como cadenas (forward references) y la función model_rebuild().
Conceptos Básicos de Referencias Circulares
¿Qué son las referencias circulares?
Una referencia circular ocurre cuando un modelo hace referencia a sí mismo o cuando dos o más modelos se referencian entre sí formando un ciclo. Por ejemplo:

Autoreferencia directa: Un modelo que contiene una referencia a su propio tipo (como un nodo en un árbol que contiene hijos del mismo tipo)
Referencia circular indirecta: Modelo A contiene una referencia a Modelo B, y Modelo B contiene una referencia a Modelo A

Problemas con las referencias circulares
En Python, las definiciones de clase se ejecutan de arriba hacia abajo, lo que crea un problema:

In [1]:
from typing import List, Optional
from pydantic import BaseModel

# Uso de string como "forward reference"
class Comment(BaseModel):
    text: str
    author: "User"  # Referencia hacia adelante como string

class User(BaseModel):
    name: str
    comments: List[Comment] = []

# Reconstruir los modelos para resolver las referencias
Comment.model_rebuild()
User.model_rebuild()

# Ahora podemos crear instancias con referencias circulares
user = User(name="María")
comment = Comment(text="Este es un comentario", author=user)
user.comments.append(comment)

print(f"Usuario: {user.name}")
print(f"Comentario: {comment.text}")
print(f"Autor del comentario: {comment.author.name}")
print(f"Número de comentarios del usuario: {len(user.comments)}")
print(f"Primer comentario del usuario: {user.comments[0].text}")

# Serialización
user_dict = user.model_dump()
print("\nSerialización (generaría un error sin manejo especial):")
print(user_dict["name"])
print(f"Comentarios: {len(user_dict['comments'])}")

Usuario: María
Comentario: Este es un comentario
Autor del comentario: María
Número de comentarios del usuario: 1
Primer comentario del usuario: Este es un comentario


ValueError: Circular reference detected (id repeated)

Solución en Pydantic v2
Pydantic v2 resuelve este problema mediante:

Referencias de cadena (forward references): Usar el nombre del tipo como una cadena
Reconstrucción del modelo: Llamar a model_rebuild() después de definir todas las clases

Referencias Circulares Avanzadas
Autoreferencias y estructuras de árbol
Un caso común de referencias circulares es la representación de estructuras jerárquicas, como árboles:

In [2]:
from typing import List, Optional
from pydantic import BaseModel, Field

class TreeNode(BaseModel):
    name: str
    children: List["TreeNode"] = []
    parent: Optional["TreeNode"] = None

# Reconstruir el modelo para resolver las referencias
TreeNode.model_rebuild()

# Crear una estructura de árbol
root = TreeNode(name="Raíz")

# Añadir hijos al nodo raíz
child1 = TreeNode(name="Hijo 1", parent=root)
child2 = TreeNode(name="Hijo 2", parent=root)
root.children = [child1, child2]

# Añadir nietos
grandchild1 = TreeNode(name="Nieto 1", parent=child1)
child1.children = [grandchild1]

# Navegar por el árbol
print(f"Raíz: {root.name}")
print(f"Hijos de la raíz: {[child.name for child in root.children]}")
print(f"Padre de '{child1.name}': {child1.parent.name}")
print(f"Hijos de '{child1.name}': {[child.name for child in child1.children]}")
print(f"Padre de '{grandchild1.name}': {grandchild1.parent.name}")
print(f"Abuelo de '{grandchild1.name}': {grandchild1.parent.parent.name}")

# Serialización personalizada para evitar recursión infinita
def serialize_tree(node, visited=None):
    if visited is None:
        visited = set()
    
    # Evitar procesamiento recursivo
    if id(node) in visited:
        return {"name": node.name, "ref": "circular"}
    
    visited.add(id(node))
    
    result = {
        "name": node.name,
        "children": [serialize_tree(child, visited) for child in node.children]
    }
    
    if node.parent:
        result["parent"] = {"name": node.parent.name}
    else:
        result["parent"] = None
    
    return result

print("\nÁrbol serializado:")
import json
print(json.dumps(serialize_tree(root), indent=2))

Raíz: Raíz
Hijos de la raíz: ['Hijo 1', 'Hijo 2']
Padre de 'Hijo 1': Raíz
Hijos de 'Hijo 1': ['Nieto 1']
Padre de 'Nieto 1': Hijo 1
Abuelo de 'Nieto 1': Raíz

Árbol serializado:
{
  "name": "Ra\u00edz",
  "children": [
    {
      "name": "Hijo 1",
      "children": [
        {
          "name": "Nieto 1",
          "children": [],
          "parent": {
            "name": "Hijo 1"
          }
        }
      ],
      "parent": {
        "name": "Ra\u00edz"
      }
    },
    {
      "name": "Hijo 2",
      "children": [],
      "parent": {
        "name": "Ra\u00edz"
      }
    }
  ],
  "parent": null
}


Referencias circulares entre múltiples modelos
Las referencias circulares pueden involucrar múltiples modelos que forman un ciclo:

In [3]:
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field, model_serializer

class Department(BaseModel):
    name: str
    director: Optional["Employee"] = None
    employees: List["Employee"] = []

class Project(BaseModel):
    title: str
    manager: Optional["Employee"] = None
    team_members: List["Employee"] = []
    client: Optional["Client"] = None

class Client(BaseModel):
    name: str
    contact_person: str
    active_projects: List[Project] = []

class Employee(BaseModel):
    name: str
    department: Optional[Department] = None
    managed_projects: List[Project] = []
    assigned_projects: List[Project] = []
    
    # Serializador personalizado para evitar recursión infinita
    @model_serializer
    def serialize_model(self) -> Dict[str, Any]:
        return {
            "name": self.name,
            "department": {"name": self.department.name} if self.department else None,
            "managed_projects": [{"title": p.title} for p in self.managed_projects],
            "assigned_projects": [{"title": p.title} for p in self.assigned_projects]
        }

# Reconstruir los modelos
Department.model_rebuild()
Project.model_rebuild()
Client.model_rebuild()
Employee.model_rebuild()

# Crear estructura de organización
eng_dept = Department(name="Ingeniería")
alice = Employee(name="Alice", department=eng_dept)
bob = Employee(name="Bob", department=eng_dept)

# Asignar director al departamento (circular)
eng_dept.director = alice
eng_dept.employees = [alice, bob]

# Crear proyecto y cliente
acme = Client(name="ACME Corp", contact_person="John Doe")
website_project = Project(
    title="Sitio Web Corporativo", 
    manager=alice,
    team_members=[alice, bob],
    client=acme
)

# Referencias circulares adicionales
alice.managed_projects = [website_project]
alice.assigned_projects = [website_project]
bob.assigned_projects = [website_project]
acme.active_projects = [website_project]

# Verificar las referencias
print(f"Departamento: {eng_dept.name}")
print(f"Director: {eng_dept.director.name}")
print(f"Proyecto: {website_project.title}")
print(f"Gerente del proyecto: {website_project.manager.name}")
print(f"Departamento del gerente: {website_project.manager.department.name}")
print(f"Cliente: {website_project.client.name}")
print(f"Proyectos activos del cliente: {[p.title for p in acme.active_projects]}")

# Usar el serializador personalizado
print("\nSerialización de empleado:")
print(alice.model_dump_json(indent=2))

Departamento: Ingeniería
Director: Alice
Proyecto: Sitio Web Corporativo
Gerente del proyecto: Alice
Departamento del gerente: Ingeniería
Cliente: ACME Corp
Proyectos activos del cliente: ['Sitio Web Corporativo']

Serialización de empleado:
{
  "name": "Alice",
  "department": {
    "name": "Ingeniería"
  },
  "managed_projects": [
    {
      "title": "Sitio Web Corporativo"
    }
  ],
  "assigned_projects": [
    {
      "title": "Sitio Web Corporativo"
    }
  ]
}


Manejo de la Serialización con Referencias Circulares
Un desafío importante con las referencias circulares es la serialización, ya que puede generar recursión infinita. Pydantic v2 ofrece varias opciones:

In [5]:
from typing import List, Optional, Dict, Any, Set
from pydantic import BaseModel, Field, model_serializer

class Person(BaseModel):
    name: str
    friends: List["Person"] = []
    
    # Método 1: Serializador personalizado con control de recursión
    @model_serializer
    def serialize_model(self) -> Dict[str, Any]:
        return {
            "name": self.name,
            "friends": [{"name": friend.name} for friend in self.friends]
        }

# Reconstruir el modelo
Person.model_rebuild()

# Crear red de amigos con referencias circulares
alice = Person(name="Alice")
bob = Person(name="Bob")
charlie = Person(name="Charlie")

alice.friends = [bob, charlie]
bob.friends = [alice, charlie]
charlie.friends = [alice, bob]

print("Red de amigos:")
print(f"{alice.name} es amiga de: {[f.name for f in alice.friends]}")
print(f"{bob.name} es amigo de: {[f.name for f in bob.friends]}")
print(f"{charlie.name} es amigo de: {[f.name for f in charlie.friends]}")

# Método 1: Usar el serializador personalizado de la clase
print("\nSerialización con @model_serializer:")
print(alice.model_dump_json(indent=2))

# Método 2: Función de serialización externa con control de ciclos
def serialize_with_cycle_detection(obj, exclude_none=True, visited=None):
    if visited is None:
        visited = set()
    
    # Si es un objeto BaseModel y ya lo visitamos
    if hasattr(obj, "__hash__") and hash(id(obj)) in visited:
        if hasattr(obj, "name"):
            return {"name": obj.name, "_is_cycle": True}
        return {"_is_cycle": True}
    
    # Marcar como visitado si es hashable
    if hasattr(obj, "__hash__"):
        visited.add(hash(id(obj)))
    
    # Si es un modelo Pydantic
    if isinstance(obj, BaseModel):
        result = {}
        for field_name, field_value in obj:
            if field_value is not None or not exclude_none:
                serialized = serialize_with_cycle_detection(
                    field_value, exclude_none, visited
                )
                result[field_name] = serialized
        return result
    
    # Si es una lista
    elif isinstance(obj, list):
        return [serialize_with_cycle_detection(item, exclude_none, visited) 
                for item in obj]
    
    # Si es un diccionario
    elif isinstance(obj, dict):
        return {
            key: serialize_with_cycle_detection(value, exclude_none, visited)
            for key, value in obj.items()
        }
    
    # Valores primitivos
    return obj

print("\nSerialización con función externa de detección de ciclos:")
import json
print(json.dumps(serialize_with_cycle_detection(alice), indent=2))

# Método 3: Usando model_dump con exclude para evitar ciertos campos
class Organization(BaseModel):
    name: str
    ceo: Optional["Employee2"] = None
    departments: List["Department2"] = []

class Department2(BaseModel):
    name: str
    organization: Optional[Organization] = None
    head: Optional["Employee2"] = None
    members: List["Employee2"] = []

class Employee2(BaseModel):
    name: str
    organization: Optional[Organization] = None
    department: Optional[Department2] = None
    reports_to: Optional["Employee2"] = None
    subordinates: List["Employee2"] = []

# Reconstruir los modelos
Organization.model_rebuild()
Department2.model_rebuild()
Employee2.model_rebuild()

# Crear estructura organizativa
company = Organization(name="TechCorp")
dev_dept = Department2(name="Desarrollo", organization=company)
hr_dept = Department2(name="RRHH", organization=company)

ceo = Employee2(name="CEO", organization=company)
dev_head = Employee2(name="Dev Lead", organization=company, department=dev_dept, reports_to=ceo)
hr_head = Employee2(name="HR Lead", organization=company, department=hr_dept, reports_to=ceo)

developer1 = Employee2(name="Developer 1", organization=company, department=dev_dept, reports_to=dev_head)
developer2 = Employee2(name="Developer 2", organization=company, department=dev_dept, reports_to=dev_head)
hr_assistant = Employee2(name="HR Assistant", organization=company, department=hr_dept, reports_to=hr_head)

# Establecer relaciones circulares
company.ceo = ceo
company.departments = [dev_dept, hr_dept]

dev_dept.head = dev_head
dev_dept.members = [dev_head, developer1, developer2]

hr_dept.head = hr_head
hr_dept.members = [hr_head, hr_assistant]

ceo.subordinates = [dev_head, hr_head]
dev_head.subordinates = [developer1, developer2]
hr_head.subordinates = [hr_assistant]

print("\nSerialización con exclude para evitar ciclos:")
# Excluir campos específicos que causan ciclos
print(company.model_dump_json(
    exclude={
        "ceo": {"organization", "department"},
        "departments": {"organization": ..., "head": {"organization", "department"}}
    },
    indent=2
))

Red de amigos:
Alice es amiga de: ['Bob', 'Charlie']
Bob es amigo de: ['Alice', 'Charlie']
Charlie es amigo de: ['Alice', 'Bob']

Serialización con @model_serializer:
{
  "name": "Alice",
  "friends": [
    {
      "name": "Bob"
    },
    {
      "name": "Charlie"
    }
  ]
}

Serialización con función externa de detección de ciclos:
{
  "name": "Alice",
  "friends": [
    {
      "name": "Bob",
      "friends": [
        {
          "name": "Alice",
          "_is_cycle": true
        },
        {
          "name": "Charlie",
          "friends": [
            {
              "name": "Alice",
              "_is_cycle": true
            },
            {
              "name": "Bob",
              "_is_cycle": true
            }
          ]
        }
      ]
    },
    {
      "name": "Charlie",
      "_is_cycle": true
    }
  ]
}

Serialización con exclude para evitar ciclos:


PydanticSerializationError: Error serializing to JSON: ValueError: Circular reference detected (id repeated)

Manejo de Ciclos de Referencia en Memoria
Un aspecto importante al usar referencias circulares es el manejo adecuado de la memoria:

In [6]:
from typing import List, Optional, Dict, Any, Set, Tuple
from pydantic import BaseModel, Field
import gc
import weakref

# Ejemplo con referencias fuertes (pueden causar memory leaks)
class NodeStrong(BaseModel):
    value: str
    children: List["NodeStrong"] = []
    model_config = {'frozen': False}  # Permitir mutabilidad

NodeStrong.model_rebuild()

# Ejemplo con referencias débiles para evitar memory leaks
class NodeWeak(BaseModel):
    value: str
    # Las referencias a padres son buenas candidatas para weakref
    parent: Optional[weakref.ReferenceType] = None
    children: List["NodeWeak"] = []
    model_config = {'frozen': False}

    def add_child(self, child: "NodeWeak") -> None:
        """Añade un hijo y establece la referencia débil al padre"""
        self.children.append(child)
        child.parent = weakref.ref(self)
    
    def get_parent(self) -> Optional["NodeWeak"]:
        """Obtiene el padre desde la referencia débil"""
        if self.parent is not None:
            return self.parent()
        return None

NodeWeak.model_rebuild()

# Test de referencias fuertes
def test_strong_refs():
    print("Creando estructura con referencias fuertes")
    root = NodeStrong(value="Root")
    
    # Crear ciclo de referencias
    child = NodeStrong(value="Child")
    root.children = [child]
    child.children = [root]  # Crear ciclo
    
    # Retornamos una referencia débil para monitorear si el objeto persiste
    return weakref.ref(root)

# Test de referencias débiles
def test_weak_refs():
    print("Creando estructura con referencias débiles")
    root = NodeWeak(value="Root")
    child = NodeWeak(value="Child")
    
    # Añadir hijo (establecerá automáticamente la referencia débil al padre)
    root.add_child(child)
    
    # Verificar la navegación
    print(f"Hijo accede al padre: {child.get_parent().value}")
    
    # Retornamos una referencia débil para monitorear
    return weakref.ref(root)

# Ejecutar pruebas
print("Test de referencias fuertes:")
root_ref_strong = test_strong_refs()

print("\nForzar recolección de basura")
gc.collect()

print(f"¿Objeto todavía en memoria? {root_ref_strong() is not None}")
print("Nota: Las referencias fuertes circulares pueden persistir hasta que el recolector de basura las detecte")

print("\nTest de referencias débiles:")
root_ref_weak = test_weak_refs()

print("\nForzar recolección de basura")
gc.collect()

print(f"¿Objeto todavía en memoria? {root_ref_weak() is not None}")
print("Nota: Los objetos con referencias débiles pueden ser liberados más fácilmente")

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class 'weakref.ReferenceType'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.10/u/schema-for-unknown-type